# Getting the data
Now let's get the FAQ data. You can run this snippet:Homework: Introduction

In this homework, we'll learn more about search and use Elastic Search for practice.

> It's possible that your answers won't match exactly. If it's the case, select the closest one.

In [1]:
from elasticsearch import Elasticsearch
from tqdm import tqdm
import requests as r

## Q1. Running Elastic
Run Elastic Search 8.4.3 and get the cluster information. If you run it on localhost, this is how you do it:

```
curl localhost:9200
```

What's the `version.build_hash` value?

In [2]:
es = Elasticsearch('http://localhost:9200')

In [3]:
print('Answer:', es.info()['version']['build_hash'])

Answer: 42f05b9372a9a4a470db3b52817899b99a76ee73


# Getting the data

Now let's get the FAQ data. You can run this snippet:

In [4]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [5]:
documents[:3]

[{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
  'section': 'General course-related questions',
  'question': 'Course - What are the prerequisites for this course?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines 

# Q2. Indexing the data

Index the data in the same way as was shown in the course videos. Make the course field a keyword and the rest should be text.

Don't forget to install the ElasticSearch client for Python:

```bash
pip install elasticsearch
```

Which function do you use for adding your data to elastic?

* insert
* index
* put
* add

Answer: index

In [6]:
index_settings = dict(
    settings=dict(number_of_shards=1, number_of_replicas=0),
    mappings=dict(
        properties=dict(
            text=dict(type='text'),
            section=dict(type='text'),
            question=dict(type='text'),
            course=dict(type='keyword')
        )
    )
)
index_name = 'course-questions'

In [7]:
es.indices.delete(index=index_name, ignore_unavailable=True)   # clear previously loaded, if any
es.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [8]:
for doc in tqdm(documents):
    es.index(index=index_name, document=doc)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:01<00:00, 731.43it/s]


## Q3. Searching

Now let's search in our index.

We will execute a query "How do I execute a command in a running docker container?".

Use only question and text fields and give question a boost of 4, and use "type": "best_fields".

What's the score for the top ranking result?

* 94.05
* 84.05
* 74.05
* 64.05

Look at the _score field.

In [9]:
query = 'How do I execute a command in a running docker container?'
query_template = dict(
    size=5,
    query=dict(
        bool=dict(
            must=dict(multi_match=dict(query=query, fields=['question^4', 'text'], type='best_fields')),
        )
    )
)

In [10]:
response = es.search(index=index_name, body=query_template)
best_score = response['hits']['hits'][0]['_score']

print('Answer:', best_score)

Answer: 82.93802


## Q4. Filtering

Now let's only limit the questions to machine-learning-zoomcamp.

Return 3 results. What's the 3rd question returned by the search engine?

* How do I debug a docker container?
* How do I copy files from a different folder into docker container’s working directory?
* How do Lambda container images work?
* How can I annotate a graph?

In [11]:
query = 'How do I execute a command in a running docker container?'
query_template = dict(
    size=3,
    query=dict(
        bool=dict(
            must=dict(multi_match=dict(query=query, fields=['question^4', 'text'], type='best_fields')),
            filter=dict(term=dict(course='machine-learning-zoomcamp'))
        )
    )
)

response = es.search(index=index_name, body=query_template)
hits = [m['_source'] for m in response['hits']['hits']]
third_question = hits[-1]['question']

print('Answer:', third_question)

Answer: How do I copy files from a different folder into docker container’s working directory?


## Q5. Building a prompt

Now we're ready to build a prompt to send to an LLM.

Take the records returned from Elasticsearch in Q4 and use this template to build the context. Separate context entries by two linebreaks (\n\n)

context_template = """
Q: {question}
A: {text}
""".strip()

Now use the context you just created along with the "How do I execute a command in a running docker container?" question to construct a prompt using the template below:

prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

What's the length of the resulting prompt? (use the len function)

* 962
* 1462
* 1962
* 2462


In [12]:
context = '\n\n'.join([f"Q:{hit['question']} A:{hit['text']}".strip() for hit in hits])
prompt = f'''
    You're a course teaching assistant.
    Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESION.

    QUESTION={query}

    CONTEXT:{context}'''

In [13]:
print('Answer:', len(prompt))

Answer: 1474


## Q6. Tokens

When we use the OpenAI Platform, we're charged by the number of tokens we send in our prompt and receive in the response.

The OpenAI python package uses tiktoken for tokenization:

```bash
pip install tiktoken
```

Let's calculate the number of tokens in our query:

```
encoding = tiktoken.encoding_for_model("gpt-4o")
```

Use the encode function. How many tokens does our prompt have?

* 122
* 222
* 322
* 422

In [14]:
import tiktoken

encoding = tiktoken.encoding_for_model('gpt-4o')

In [15]:
encoded_prompt = encoding.encode(prompt)

In [16]:
print('Answer:', len(encoded_prompt), 'tokens')

Answer: 330 tokens
